In [1]:
# reload modules in .py files
%load_ext autoreload
%autoreload 2

In [2]:
# pull repo
!git clone https://github.com/char-tan/sparsity

Cloning into 'sparsity'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 61 (delta 27), reused 42 (delta 13), pack-reused 0
Unpacking objects: 100% (61/61), done.


In [3]:
# change working directory
import os
os.chdir('sparsity/ops')

In [16]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing ops_cpp.egg-info/PKG-INFO
writing dependency_links to ops_cpp.egg-info/dependency_links.txt
writing top-level names to ops_cpp.egg-info/top_level.txt
/usr/local/lib/python3.8/dist-packages/torch/utils/cpp_extension.py:476: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
writing manifest file 'ops_cpp.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
/usr/local/lib/python3.8/dist-packages/torch/utils/cpp_extension.py:387: UserWarning: The detected CUDA version (11.2) has a minor version mismatch with the version that was used to compile PyTorch (11.6). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
building 'ops_cpp' extension
x86_64-linux

In [5]:
# Using -e has caveats: https://stackoverflow.com/a/20043907
!pip3 install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/sparsity/ops
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for ops-cpp: filename=ops_cpp-0.0.0-cp38-cp38-linux_x86_64.whl size=2407560 sha256=6a713393dfd7a9fe6282f56489796e06ef9d373ef78b2d939a559fdfd0e9a5c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-ebuygvg3/wheels/a0/60/38/cf329a695010b267296cd00c31c2a180e9d57ca424280b3d28
Successfully built ops-cpp
  Attempting uninstall: ops-cpp
    Found existing installation: ops-cpp 0.0.0
    Uninstalling ops-cpp-0.0.0:
      Successfully uninstalled ops-cpp-0.0.0

### Sample usage of our extension

In [10]:
import torch
import os, sys
sys.path.append(os.getcwd())
import ops_cpp

a = torch.rand([3,3]).cuda()
b = a
c = ops_cpp.add(a, b)
print(c)

tensor([[0.6803, 1.8560, 1.7057],
        [0.9444, 1.2852, 1.8762],
        [0.1864, 0.8852, 0.7403]], device='cuda:0')


In [17]:
!git diff

diff --git a/ops/ops_cuda_kernel.cu b/ops/ops_cuda_kernel.cu
index 092ed31..2958650 100644
--- a/ops/ops_cuda_kernel.cu
+++ b/ops/ops_cuda_kernel.cu
@@ -4,10 +4,33 @@
 #include <cuda_runtime.h>
 
 
+template <typename scalar_t>
+__global__ void cuda_add_kernel(
+    scalar_t* __restrict__ a,
+    scalar_t* __restrict__ b,
+    scalar_t* __restrict__ c
+    ) {
+        c = a + b
+      }
+
+
 torch::Tensor cuda_add(
 		torch::Tensor a,
 		torch::Tensor b)
 {
-	auto c = torch::add(a, b);
-	return c;
+
+  auto c = torch::zeros_like(a);
+
+  AT_DISPATCH_FLOATING_TYPES(a.type(), "cuda_add", ([&] {
+    cuda_add_kernel<scalar_t><<<1, 1>>>(
+        a.data<scalar_t>(),
+        b.data<scalar_t>(),
+        c.data<scalar_t>(),
+  })));
+
+  return {c}
 }
+
+
+
+
